In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
import pickle

In [2]:
current_year = 2023

In [3]:
market = Market()
spec = ADatabase("comet")
umod = UniversalModeler()

In [4]:
factors = [str(x) for x in range(14)]

In [5]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [6]:
included_columns.extend(factors)

In [7]:
training_sets = []

In [12]:
ticker = "BTC"
try:
    prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
    prices["ticker"] = ticker
    prices = p.column_date_processing(prices)
    prices["year"] = [x.year for x in prices["date"]]
    prices["quarter"] = [x.quarter for x in prices["date"]]
    ticker_data = prices[prices["ticker"]==ticker]
    ticker_data.sort_values("date",ascending=True,inplace=True)
    ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
    ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
    for i in range(14):
        ticker_data[str(i)] = ticker_data["adjclose"].shift(i)
    ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
    ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
    ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
    ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
    ticker_data.dropna(inplace=True)
    ticker_data["ticker"] = ticker
    ticker_data["y"] = ticker_data["adjclose"].shift(-1)
    ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
    ticker_data.dropna(inplace=True)
    ticker_data = ticker_data[included_columns]
    training_sets.append(ticker_data)
except Exception as e:
    print(ticker,str(e))

In [13]:
data = pd.concat(training_sets)

In [14]:
data = data.dropna()

In [15]:
spec.connect()
for modeler in [umod]:
    for year in tqdm(range(current_year,current_year+1)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - 4)].reset_index(drop=True)
        set_name = "models"
        stuff = modeler.recommend_model(training_slice,factors)
spec.disconnect()

  0%|                                                                | 0/1 [00:00<?, ?it/s]

Epoch 1/10
10/10 [==============================] - 1s 1ms/step - loss: 171745536.0000 - mean_absolute_percentage_error: 36.5060
Epoch 2/10
10/10 [==============================] - 0s 1ms/step - loss: 48786784.0000 - mean_absolute_percentage_error: 19.2369
Epoch 3/10
10/10 [==============================] - 0s 1ms/step - loss: 23108068.0000 - mean_absolute_percentage_error: 12.6165
Epoch 4/10
10/10 [==============================] - 0s 1ms/step - loss: 16120195.0000 - mean_absolute_percentage_error: 10.7009
Epoch 5/10
10/10 [==============================] - 0s 889us/step - loss: 12605833.0000 - mean_absolute_percentage_error: 9.7372
Epoch 6/10
10/10 [==============================] - 0s 889us/step - loss: 10867001.0000 - mean_absolute_percentage_error: 8.9052
Epoch 7/10
10/10 [==============================] - 0s 1ms/step - loss: 10535926.0000 - mean_absolute_percentage_error: 9.3151
Epoch 8/10
10/10 [==============================] - 0s 1ms/step - loss: 10588029.0000 - mean_absolute_

100%|████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.86s/it]


In [16]:
stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_absolute_percentage_error
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-05-12 09:05:49         2522
metadata.json                                  2023-05-12 09:05:49           64
variables.h5                                   2023-05-12 09:05:49        84160


In [17]:
spec.connect()
models = spec.store("models",stuff)
spec.disconnect()